In [10]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import itertools
import csv
import json
from os import listdir, system, path


In [11]:

#Lista todos los archivos al interior de las carpetas 
KeysIndoor = []
for dirr in list(filter(  path.isdir  , ["DataIndoor/" + r for r in listdir("DataIndoor")] )):
    for i in listdir(dirr):
        if(i[-5:] == ".json"):
            KeysIndoor.append("{}/{}".format(dirr,i))

KeysOutdoor = []
for dirr in list(filter(  path.isdir  , ["DataOutdoor/" + r for r in listdir("DataOutdoor")] )):
    for i in listdir(dirr):
        if(i[-5:] == ".json"):
            KeysOutdoor.append("{}/{}".format(dirr,i))


In [12]:
"Crea archivo de llaves para mas adelante"
with open("DataIndoor/keys.csv", 'w') as File:
    for i in KeysIndoor: File.write(i+"\n")
with open("DataOutdoor/keys.csv", 'w') as File:
    for i in KeysOutdoor: File.write(i+"\n")

In [13]:
def reparar_json(keyBads): #Debe ingresar llaves limpias, tipo string sin comillas 
    with open(keyBads, 'r') as File:  
        reader = File.read()
        recorte = reader[::-1].find("{,") + 2
        newdata= reader[:-recorte] + "]}"
    with open(key, 'w') as File:
        File.write(newdata)

def putJson(Keys,Data):  #Une los archivos json separados en uno solo, igual al descargado
    for key in Keys:
        try:
            with open(key, mode= "r") as File:
                data = json.load(File)            
                for i in data["mediciones"]:
                    Data[0]["mediciones"].append(i)
        except:
            try:
                reparar_json(key)
                with open(key, mode= "r") as File:
                    data = json.load(File)                    
                    for i in data["mediciones"]:
                        Data[0]["mediciones"].append(i)
            except:
                pass
    return Data

In [14]:
DataIndoor =  [{"_id":"5fcdb571be37d902bc84ae35","id_dataset":"5fcdb571be37d902bc84ae35","version":"1.0","id_estacion":"mc_indoor","gps":False,"timezone":"UTC-5","mediciones":[]}]
DataOutdoor = [{"_id":"5fcdb571be37d902bc84ae35","id_dataset":"5fcdb571be37d902bc84ae35","version":"1.0","id_estacion":"mc_outdoor","gps":False,"timezone":"UTC-5","mediciones":[]}]

#Desde aca se repite el codigo de Download
r_json_mc_indoor  = putJson(KeysIndoor,DataIndoor)
r_json_mc_outdoor = putJson(KeysOutdoor,DataOutdoor)


In [15]:
data_json_mc_indoor = [i for pack in r_json_mc_indoor for i in pack["mediciones"]]
data_json_mc_outdoor = [i for pack in r_json_mc_outdoor for i in pack["mediciones"]]
data_mc_indoor_ = json_normalize(data_json_mc_indoor)
data_mc_outdoor_ = json_normalize(data_json_mc_outdoor)

In [16]:
columns= list(data_mc_indoor_["id_parametro"].drop_duplicates())
columns.append("date_time")
data_mc_indoor = data_mc_indoor_.pivot_table(index='fecha_hora_med', columns='id_parametro')['valor']
data_mc_outdoor = data_mc_outdoor_.pivot_table(index='fecha_hora_med', columns='id_parametro')['valor']

data_mc_indoor.to_csv("../indoor.csv", sep = ",")
data_mc_outdoor.to_csv("../outdoor.csv", sep = ",")
data_mc_outdoor

id_parametro,b,co2__desv,co2_adc_1,co2_adc_1_desv,co2_adc_1_max,co2_adc_1_min,co2_adc_2,co2_adc_2_desv,co2_adc_2_max,co2_adc_2_min,...,pm_n_1_0,pm_n_1_0_desv,pm_n_1_0_max,pm_n_1_0_min,pm_n_2_5,pm_n_2_5_desv,pm_n_2_5_max,pm_n_2_5_min,t,v
fecha_hora_med,,,,,,,,,,,,,,,,,,,,,
2020-11-24T00:19:57Z,3.774871,NaN,1685.8670,12.099400,1715.0,1672.0,879.0000,11.617470,903.0,864.0,...,107.73330,11.506920,133.0,86.0,6.266667,1.799106,10.0,4.0,NaN,0.000000
2020-11-24T00:20:56Z,NaN,NaN,1668.0330,2.235811,1675.0,1664.0,861.7000,2.380235,866.0,856.0,...,98.26667,15.534020,127.0,81.0,6.066667,4.940567,14.0,0.0,27.12150,NaN
2020-11-24T00:22:02Z,NaN,NaN,1670.5000,3.421534,1676.0,1665.0,859.1000,1.768498,865.0,857.0,...,87.40000,14.262560,107.0,62.0,7.066667,3.841097,13.0,0.0,NaN,NaN
2020-11-24T00:23:02Z,3.613984,NaN,1678.4330,1.924136,1682.0,1674.0,858.4333,1.006302,860.0,856.0,...,87.03333,7.378829,103.0,73.0,6.800000,2.006885,10.0,4.0,27.42200,0.000000
2020-11-24T00:25:31Z,3.739936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2020-11-24T00:27:12Z,NaN,NaN,1680.4000,2.283373,1687.0,1676.0,854.4333,1.869600,860.0,852.0,...,82.65000,21.283250,165.0,48.0,5.050000,1.789155,13.0,0.0,NaN,NaN
2020-11-24T00:28:18Z,NaN,NaN,1680.0000,1.364576,1684.0,1676.0,854.7667,1.165106,858.0,853.0,...,109.40000,11.684350,129.0,84.0,8.600000,5.709036,22.0,0.0,26.37950,NaN
2020-11-24T00:29:24Z,3.759242,NaN,1680.5330,0.819307,1683.0,1679.0,854.2000,1.297212,856.0,852.0,...,122.26670,9.695124,142.0,106.0,8.266666,2.715642,14.0,4.0,NaN,0.000000
2020-11-24T00:31:37Z,3.741774,NaN,1675.3330,1.347624,1678.0,1673.0,853.3000,1.118805,855.0,849.0,...,110.61670,8.263691,163.0,83.0,7.783333,1.942606,17.0,2.0,NaN,0.000306


In [17]:
for i in data_mc_outdoor.columns:
    print(i)

b
co2__desv
co2_adc_1
co2_adc_1_desv
co2_adc_1_max
co2_adc_1_min
co2_adc_2
co2_adc_2_desv
co2_adc_2_max
co2_adc_2_min
co2_desv
co2_ppm_1
co2_ppm_1_deesv
co2_ppm_1_desv
co2_ppm_1_max
co2_ppm_1_min
co2_ppm_2
co2_ppm_2_desv
co2_ppm_2_max
co2_ppm_2_min
co2_rdesv
co2_res_1
co2_res_1_desv
co2_res_1_max
co2_res_1_min
co2_res_2
co2_res_2_desv
co2_res_2_max
co2_res_2_min
co2_resco2_ppm_1_max
co2_resv
co2_rzer
co2_rzero
co2_rzero_1
co2_rzero_1_desv
co2_rzero_1_max
co2_rzero_1_min
co2_rzero_2
co2_rzero_2_desv
co2_rzero_2_max
co2_rzero_2_min
co2_rzero_2_mv
co2_rzerv
co2_rzesv
co2_rzev
co2_rzsv
h
p
pm10_0_au
pm10_0_au_desv
pm10_0_au_max
pm10_0_au_min
pm10_0_sp
pm10_0_sp_desv
pm10_0_sp_max
pm10_0_sp_min
pm1_0_au
pm1_0_au_desv
pm1_0_au_max
pm1_0_au_min
pm1_0_sp
pm1_0_sp_desv
pm1_0_sp_max
pm1_0_sp_min
pm2_5_au
pm2_5_au_desv
pm2_5_au_max
pm2_5_au_min
pm2_5_sp
pm2_5_sp_desv
pm2_5_sp_max
pm2_5_sp_min
pm_n_0_3
pm_n_0_3_desv
pm_n_0_3_max
pm_n_0_3_min
pm_n_0_5
pm_n_0_5_desv
pm_n_0_5_max
pm_n_0_5_min
pm_n_10